# Pre-processing Aggregated Amounts data for WaDE upload.
Purpose:  To pre-process the aggregated amounts data into one master file for simple DataFrame creation and extraction

In [1]:
# Needed Libraries / Modules

# ---- working with data ----
import os  # native operating system interaction
import numpy as np  # mathematical array manipulation
import pandas as pd  # data structure and data analysis
import geopandas as gpd  # geo-data structure and data analysis

# ---- visualization ----
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library

# ---- API data retrieval ----
import requests  # http requests
import json  # JSON parse

# ---- Cleanup ----
import re  # string regular expression manipulation
from datetime import datetime  # date and time manipulation
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x)  # suppress scientific notation in Pandas

In [ ]:
# ---- working directory ----
workingDirString = #"G:/Shared drives/WaDE Data/NorthDakota/AggregatedAmounts" # set working directory folder string here
os.chdir(workingDirString)
print(f'The working Directory is:', workingDirString)

## Input Source Data

In [3]:
# Input File (shp or csv)
inputFile ="RawInputData/{enter file name here}.zip"
dfIn = gpd.read_file(inputFile, encoding = "ISO-8859-1").replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfIn:
    df['WaDEUUID'] = "ndag" + dfIn.index.astype(str)
    dfIn.to_csv('RawInputData/{enter file name here}.zip', compression=dict(method='zip', archive_name='enter file name here}.csv'), index=False)

print(len(dfIn))
dfIn.head()

In [9]:
# Transfer input data to WaDE specific output

# create dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfIn['WaDEUUID']

# Method Info
df['in_MethodUUID'] = ""

# Variable Info
df['in_VariableSpecificUUID'] = "" # determine below
df['in_AggregationIntervalUnitCV'] = ""
df['in_VariableCV'] = ""
df['in_VariableSpecificCV'] = "" # determine below

# Organization Info
df['in_OrganizationUUID'] = ""

# Water Source
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df["in_WaterSourceName"] = ""
df['in_WaterSourceNativeID'] = ""
df["in_WaterSourceTypeCV"] = ""

# ReportingUnits Info
df['in_EPSGCodeCV'] = 4326
df['in_ReportingUnitName'] = ""
df['in_ReportingUnitNativeID'] = ""
df['in_ReportingUnitProductVersion'] = ""
df['in_ReportingUnitTypeCV'] = "HUC8"
df['in_ReportingUnitUpdateDate'] = ""
df['in_StateCV'] = ""

# AggregatedAmounts Info
df['in_AllocationCropDutyAmount'] = ""
df['in_Amount'] = ""
df['in_BeneficialUseCategory'] = ""
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_InterbasinTransferFromID'] = ""
df['in_InterbasinTransferToID'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerGeneratedGWh'] = ""
df['in_PowerType'] = ""
df['in_PrimaryUseCategoryCV'] = ""
df['in_ReportYearCV'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_TimeframeEnd'] = ""
df['in_TimeframeStart'] = ""

outdf = df.copy()
outdf = outdf.drop_duplicates().reset_index(drop=True)
print(len(outdf))
outdf.head(5)

## Concatenate Data (if needed).

In [ ]:
# # Concatenate dataframes

# frames = [df1, df2]  # list all out dataframes here
# outdf = pd.concat(frames)
# outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
# print(len(outdf))

## Clean Data / Data Types

In [10]:
# Clean name entries of spcial characters
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;/\)(-]", "", Val).title().replace("  ", " ").strip().rstrip(',')
    return Val

In [12]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [11]:
outdf['in_ReportingUnitName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_ReportingUnitName']), axis=1)
outdf['in_ReportingUnitName'].unique()

In [13]:
# Ensure Empty String / remove string value of "nan"

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [14]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [15]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [16]:
outdf['in_ReportingUnitName'] = outdf.apply(lambda row: ensureEmptyString(row['in_ReportingUnitName']), axis=1)
outdf['in_ReportingUnitName'].unique()

In [17]:
outdf['in_ReportingUnitTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_ReportingUnitTypeCV']), axis=1)
outdf['in_ReportingUnitTypeCV'].unique()

In [18]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
uniqueList = list(set([i.strip() for i in ','.join(outdf['in_BeneficialUseCategory'].astype(str)).split(',')]))
uniqueList.sort()
uniqueList

In [19]:
# Fixing n_Amount entry is either numireic or a 0

outdf['in_Amount'] = pd.to_numeric(outdf['in_Amount'], errors='coerce').round(2).replace(0,"").fillna("")
outdf['in_Amount'].unique()

In [20]:
# Creating WaDE Custom water source native ID for easy water source identification
# use unique WaterSourceName and WaterSourceType values
# ----------------------------------------------------------------------------------------------------

# Create temp in_WaterSourceNativeID dataframe of unique water source.
def assignIdValueFunc(colRowValue):
    string1 = str(colRowValue)
    outstring = "wadeId" + string1
    return outstring

dfTempID = pd.DataFrame()
dfTempID['in_WaterSourceName'] = outdf['in_WaterSourceName'].astype(str).str.strip()
dfTempID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV'].astype(str).str.strip()
dfTempID = dfTempID.drop_duplicates()

dfTempCount = pd.DataFrame(index=dfTempID.index)
dfTempCount["Count"] = range(1, len(dfTempCount.index) + 1)
dfTempID['in_WaterSourceNativeID'] = dfTempCount.apply(lambda row: assignIdValueFunc(row['Count']), axis=1)
dfTempID['linkKey'] = dfTempID['in_WaterSourceName'].astype(str) + dfTempID['in_WaterSourceTypeCV'].astype(str)
IdDict = pd.Series(dfTempID.in_WaterSourceNativeID.values, index=dfTempID.linkKey.astype(str)).to_dict()
# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom site native ID
def retrieveIdValueFunc(checkVal, valA, valB):
    checkVal = str(checkVal).strip()
    if checkVal == "":
        linkKeyVal = str(valA).strip() + str(valB).strip()
        outString = IdDict[linkKeyVal]
    else:
        outString = checkVal
    return outString

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveIdValueFunc(row['in_WaterSourceNativeID'], 
                                                                              row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

In [21]:
# determine in_VariableSpecificCV

outdf['in_VariableSpecificCV'] = outdf['in_VariableCV'].astype(str) + "_" + outdf['in_AggregationIntervalUnitCV'].astype(str) + "_" + outdf['in_BeneficialUseCategory'].astype(str) + "_" + outdf['in_WaterSourceTypeCV'].astype(str)
outdf['in_VariableSpecificCV'].unique()

In [22]:
%%time

# Creating WaDE VariableSpecificUUID for easy VariableSpecificCV identification 
# use these inputs: VariableCV_AggregationIntervalUnitCV_BeneficalUse_WaterSourceTypeCV
# ----------------------------------------------------------------------------------------------------

# Create temp VariableSpecificUUID dataframe of unique water source.
def assignVariableSpecificUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "NDag_V" + string1
    return outstring

dfVariableSpecificUUID = pd.DataFrame()
dfVariableSpecificUUID['in_VariableCV'] = outdf['in_VariableCV']
dfVariableSpecificUUID['in_AggregationIntervalUnitCV'] = outdf['in_AggregationIntervalUnitCV']
dfVariableSpecificUUID['in_BeneficialUseCategory'] = outdf['in_BeneficialUseCategory']
dfVariableSpecificUUID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfVariableSpecificUUID = dfVariableSpecificUUID.drop_duplicates()

dftemp = pd.DataFrame(index=dfVariableSpecificUUID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfVariableSpecificUUID['in_VariableSpecificUUID'] = dftemp.apply(lambda row: assignVariableSpecificUUID(row['Count']), axis=1)
dfVariableSpecificUUID['linkKey'] = dfVariableSpecificUUID['in_VariableCV'].astype(str) + dfVariableSpecificUUID['in_AggregationIntervalUnitCV'].astype(str) + dfVariableSpecificUUID['in_BeneficialUseCategory'].astype(str) + dfVariableSpecificUUID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
VariableSpecificUUIDdict = pd.Series(dfVariableSpecificUUID.in_VariableSpecificUUID.values, index=dfVariableSpecificUUID.linkKey.astype(str)).to_dict()
def retrieveVariableSpecificUUID(A, B, C, D):
    if (A == '' and B == '' and C == '' and D == '') or (pd.isnull(A) and pd.isnull(B) and pd.isnull(C) and pd.isnull(D)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip() + str(C).strip() + str(D).strip()
        try:
            outList = VariableSpecificUUIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_VariableSpecificUUID'] = outdf.apply(lambda row: retrieveVariableSpecificUUID(row['in_VariableCV'], row['in_AggregationIntervalUnitCV'], row['in_BeneficialUseCategory'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_VariableSpecificUUID'].unique()

## Shapefile Data
- For attaching geometry to csv inputs.

In [23]:
# Shapefile Data (can just be copy of above input)\

shapefileInput = gpd.read_file('RawInputData/shapefiles/{enter file name here}.zip')
dfshapetemp = gpd.read_file(shapefileInput)
print(len(dfshapetemp))
dfshapetemp.head()

In [24]:
columnsList = ['in_ReportingUnitNativeID', 'geometry']
outshape = pd.DataFrame(columns=columnsList)
outshape['in_ReportingUnitNativeID'] = "" # chould match above 'in_ReportingUnitNativeID' if file not copied
outshape['geometry'] = dfshapetemp['geometry']
outshape = outshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
outshape.head(20)

# Export Data

In [25]:
outdf.info()

In [26]:
outdf

In [27]:
# Export the output dataframe
outdf.to_csv('RawInputData/Pag_Main.zip', compression=dict(method='zip', archive_name='Pag_Main.csv'), index=False)   # The output, save as a zip
outshape.to_csv('RawInputData/P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.